# Extract surface fields from the SUNTANS model output
## Dask worker setup instructions

https://github.com/mrayson/pawsey-containers/tree/master/hpc-python/jupyter-sfoda

These settings do the job:

 1. `dask-scheduler --scheduler-file scheduler-$HOSTNAME.json --idle-timeout 0`
 2. `ssh localhost "cd /software/projects/pawsey0106/mrayson/projects && sbatch --ntasks=32 -c 8 start_worker.slm scheduler-$HOSTNAME.json ./"`


In [1]:
hostname = !echo $(hostname)
hostname[0]

'nid002054'

In [2]:
import os
from distributed import Client
client=Client(scheduler_file='../../scheduler-' + hostname[0] + '.json')
client

<Client: 'tcp://10.253.133.97:8786' processes=32 threads=256, memory=429.92 GiB>

In [3]:
client.restart() # This clears all data

<Client: 'tcp://10.253.133.97:8786' processes=32 threads=256, memory=429.92 GiB>

In [4]:
import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from s3fs import S3FileSystem, S3Map

import dask.array as da
import dask

from sfoda.suntans import sunxarray as sxr
from sfoda.utils.myproj import MyProj

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ln_h6pv6 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Dask array approach (works)

In [5]:
def load_surface_variables(runnum, dzz=None, rho0=None, timechunk=10):
    # Get the file names
    print(72*'#')
    print('\t File set: ',runnum)
    basedir =  '/scratch/pawsey0106/mrayson//SUNTANS_NWS_2km/NWS_2km_GLORYS_hex_Nk80dt60_2013_2014_data{}/'.format(runnum)
    ncfiles = '{}/*.nc.*'.format(basedir)
    filenames = sorted(glob.glob(ncfiles))
    print(filenames[0], filenames[-1])
    
    # Open all of the files (drop the nan values)
    #ds_all = [xr.open_dataset(ff, chunks={}).fillna(0) for ff in filenames]
    ds_all = [xr.open_dataset(ff, chunks={}) for ff in filenames]

    
    # Find the ghost cells
    mnptr = xr.DataArray(da.concatenate([ds['mnptr'] for ds in ds_all], axis=0))
    ghost = sxr.find_ghost_cells(mnptr)
    Nc = ghost.shape[0]
    
    eta = da.concatenate([ds['eta'] for ds in ds_all],axis=1)[...,ghost].persist()
    tau_x = da.concatenate([ds['tau_x'] for ds in ds_all],axis=1)[...,ghost].persist()
    tau_y = da.concatenate([ds['tau_y'] for ds in ds_all],axis=1)[...,ghost].persist()
    
    uc = da.concatenate([ds['uc'].isel(Nk=0) for ds in ds_all],axis=1)[...,ghost].persist()
    vc = da.concatenate([ds['vc'].isel(Nk=0) for ds in ds_all],axis=1)[...,ghost].persist()
    temp = da.concatenate([ds['temp'].isel(Nk=0) for ds in ds_all],axis=1)[...,ghost].persist()
    nu_v = da.concatenate([ds['nu_v'].isel(Nk=0) for ds in ds_all],axis=1)[...,ghost].persist()
  
    # Load all of the grid variables
    xv = da.concatenate([ds['xv'] for ds in ds_all],axis=0)[ghost]
    yv= da.concatenate([ds['yv'] for ds in ds_all],axis=0)[ghost]
    nfaces= da.concatenate([ds['nfaces'] for ds in ds_all],axis=0)[ghost]
    dv= da.concatenate([ds['dv'] for ds in ds_all],axis=0)[ghost]
    cells= da.concatenate([ds['cells'] for ds in ds_all],axis=0)[ghost,...]

    xp = ds_all[0]['xp']
    yp = ds_all[0]['yp']
    z_r = ds_all[0]['z_r']
    dz = ds_all[0]['dz']
    time = ds_all[0]['time']
        
    # Convert the coordinates
    P=MyProj('merc')
    lonv, latv = P.to_ll(xv, yv)
    lonp, latp = P.to_ll(xp, yp)

    # Create a new dataset
    dsout = xr.Dataset(
        {
        'xv':xr.DataArray(xv, dims=('Nc',), attrs=ds_all[0]['xv'].attrs),
        'yv':xr.DataArray(yv, dims=('Nc',), attrs=ds_all[0]['yv'].attrs),
        'xp':xr.DataArray(xp, dims=('Np',), attrs=ds_all[0]['xp'].attrs),
        'yp':xr.DataArray(yp, dims=('Np',), attrs=ds_all[0]['yp'].attrs),
        'lonv':xr.DataArray(lonv, dims=('Nc',), attrs=ds_all[0]['xv'].attrs),
        'latv':xr.DataArray(latv, dims=('Nc',), attrs=ds_all[0]['yv'].attrs),
        'lonp':xr.DataArray(lonp, dims=('Np',), attrs=ds_all[0]['xp'].attrs),
        'latp':xr.DataArray(latp, dims=('Np',), attrs=ds_all[0]['yp'].attrs),
        'dv':xr.DataArray(dv, dims=('Nc',), attrs=ds_all[0]['dv'].attrs),
        'dz':xr.DataArray(dz, dims=('Nk',), attrs=ds_all[0]['dz'].attrs),
        'cells':xr.DataArray(cells, dims=('Nc','numsides'), attrs=ds_all[0]['cells'].attrs),
        'nfaces':xr.DataArray(nfaces, dims=('Nc',), attrs=ds_all[0]['nfaces'].attrs),    
        'time':xr.DataArray(time, dims=('time',), attrs=ds_all[0]['time'].attrs),
        'eta':xr.DataArray(eta, dims=('time','Nc'), attrs=ds_all[0]['eta'].attrs),
        'tau_x':xr.DataArray(eta, dims=('time','Nc'), attrs=ds_all[0]['tau_x'].attrs),
        'tau_y':xr.DataArray(eta, dims=('time','Nc'), attrs=ds_all[0]['tau_y'].attrs),
        'uc':xr.DataArray(uc, dims=('time','Nc'), attrs=ds_all[0]['uc'].attrs),
        'vc':xr.DataArray(vc, dims=('time','Nc'), attrs=ds_all[0]['vc'].attrs),
        'temp':xr.DataArray(temp, dims=('time','Nc'), attrs=ds_all[0]['temp'].attrs),
        'nu_v':xr.DataArray(temp, dims=('time','Nc'), attrs=ds_all[0]['nu_v'].attrs),

        },
        attrs={'Description':'SUNTANS surface fields',
              'Author':os.environ['USER'],
              })
    return dsout


In [6]:
outfile = '/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/NWS_2km_GLORYS_2013_2014_surface_v2.zarr'

mode='w'
append_dim=None

for ii in range(1,13):
    dsout = load_surface_variables(ii)
    print('\tWriting to zarr file: ', outfile)
    #dsout.chunk({'Nc':-1,'time':1}).to_zarr(outfile, mode=mode, append_dim=append_dim, compute=True)
    dsout.chunk({'Nc':2000}).to_zarr(outfile, mode=mode, append_dim=append_dim, compute=True)

    mode='a'
    append_dim='time'
    
    # Clear memory
    client.restart()
    break

########################################################################
	 File set:  1
/scratch/pawsey0106/mrayson//SUNTANS_NWS_2km/NWS_2km_GLORYS_hex_Nk80dt60_2013_2014_data1/NWS_2km_GLORYS_hex_20130701.nc.0 /scratch/pawsey0106/mrayson//SUNTANS_NWS_2km/NWS_2km_GLORYS_hex_Nk80dt60_2013_2014_data1/NWS_2km_GLORYS_hex_20130701.nc.99
	Writing to zarr file:  /scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/NWS_2km_GLORYS_2013_2014_surface_v2.zarr


# Easy to code approach... doesn't work :(

In [5]:
runnum=1
basedir =  '/scratch/pawsey0106/mrayson//SUNTANS_NWS_2km/NWS_2km_GLORYS_hex_Nk80dt60_2013_2014_data{}/'.format(runnum)
ncfiles = '{}/*.nc.*'.format(basedir)
filenames = sorted(glob.glob(ncfiles))
filenames[0]

'/scratch/pawsey0106/mrayson//SUNTANS_NWS_2km/NWS_2km_GLORYS_hex_Nk80dt60_2013_2014_data1/NWS_2km_GLORYS_hex_20130701.nc.0'

In [7]:
ds = sxr.open_parallel(filenames)
ds

/opt/conda/lib/python3.10/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.Dataset>
Dimensions:       (Nc: 230195, numsides: 8, Np: 454016, Nk: 80, Nkw: 81,
                   time: 745)
Coordinates:
    Nk            (Nc) int32 dask.array<chunksize=(973,), meta=np.ndarray>
    xv            (Nc) float64 dask.array<chunksize=(973,), meta=np.ndarray>
    yv            (Nc) float64 dask.array<chunksize=(973,), meta=np.ndarray>
    z_r           (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
    z_w           (Nkw) float64 dask.array<chunksize=(81,), meta=np.ndarray>
  * time          (time) datetime64[ns] 2013-07-01T00:01:00 ... 2013-08-01
Dimensions without coordinates: Nc, numsides, Np, Nkw
Data variables: (12/36)
    suntans_mesh  int32 -2147483647
    cells         (Nc, numsides) int32 dask.array<chunksize=(973, 8), meta=np.ndarray>
    face          (Nc, numsides) int32 dask.array<chunksize=(973, 8), meta=np.ndarray>
    nfaces        (Nc) int32 dask.array<chunksize=(973,), meta=np.ndarray>
    neigh         (Nc, numsides) int32 dask.array<chunksize=(973, 8), meta=np.ndarray>
    mnptr         (Nc) int32 dask.array<chunksize=(973,), meta=np.ndarray>
    ...            ...
    Hl            (time, Nc) float64 dask.array<chunksize=(745, 973), meta=np.ndarray>
    Hlw           (time, Nc) float64 dask.array<chunksize=(745, 973), meta=np.ndarray>
    Hsw           (time, Nc) float64 dask.array<chunksize=(745, 973), meta=np.ndarray>
    tau_x         (time, Nc) float64 dask.array<chunksize=(745, 973), meta=np.ndarray>
    tau_y         (time, Nc) float64 dask.array<chunksize=(745, 973), meta=np.ndarray>
    EP            (time, Nc) float64 dask.array<chunksize=(745, 973), meta=np.ndarray>
Attributes:
    title:    SUNTANS NetCDF output file

In [26]:
droplist = ['EP','cloud','RH','rain','Pair','Tair','Vwind','Uwind','w','dzz']
# Subset the top layer, drop the first time step, and drop unwanted variables
ds_surface = ds.isel(Nk=0, time=slice(1,None)).drop_vars(droplist)#.chunk({'Nc':2000})
ds_surface

<xarray.Dataset>
Dimensions:       (Nc: 230195, numsides: 8, Np: 454016, Nkw: 81, time: 744)
Coordinates:
    Nk            (Nc) int32 dask.array<chunksize=(973,), meta=np.ndarray>
    xv            (Nc) float64 dask.array<chunksize=(973,), meta=np.ndarray>
    yv            (Nc) float64 dask.array<chunksize=(973,), meta=np.ndarray>
    z_r           float64 dask.array<chunksize=(), meta=np.ndarray>
    z_w           (Nkw) float64 dask.array<chunksize=(81,), meta=np.ndarray>
  * time          (time) datetime64[ns] 2013-07-01T01:00:00 ... 2013-08-01
Dimensions without coordinates: Nc, numsides, Np, Nkw
Data variables: (12/26)
    suntans_mesh  int32 -2147483647
    cells         (Nc, numsides) int32 dask.array<chunksize=(973, 8), meta=np.ndarray>
    face          (Nc, numsides) int32 dask.array<chunksize=(973, 8), meta=np.ndarray>
    nfaces        (Nc) int32 dask.array<chunksize=(973,), meta=np.ndarray>
    neigh         (Nc, numsides) int32 dask.array<chunksize=(973, 8), meta=np.ndarray>
    mnptr         (Nc) int32 dask.array<chunksize=(973,), meta=np.ndarray>
    ...            ...
    Hs            (time, Nc) float64 dask.array<chunksize=(744, 973), meta=np.ndarray>
    Hl            (time, Nc) float64 dask.array<chunksize=(744, 973), meta=np.ndarray>
    Hlw           (time, Nc) float64 dask.array<chunksize=(744, 973), meta=np.ndarray>
    Hsw           (time, Nc) float64 dask.array<chunksize=(744, 973), meta=np.ndarray>
    tau_x         (time, Nc) float64 dask.array<chunksize=(744, 973), meta=np.ndarray>
    tau_y         (time, Nc) float64 dask.array<chunksize=(744, 973), meta=np.ndarray>
Attributes:
    title:    SUNTANS NetCDF output file

In [27]:
outfile = '/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/NWS_2km_GLORYS_2013_2014_surface_v2.zarr'
outfile = '/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/NWS_2km_GLORYS_2013_2014_surface_v2.nc'

mode='w'
append_dim=None
# ds_surface.to_zarr(outfile, mode=mode, append_dim=append_dim, compute=True)
ds_surface.to_netcdf(outfile, mode=mode, compute=True)

# for ii in range(1,13):
#     dsout = load_surface_variables(ii)
#     print('\tWriting to zarr file: ', outfile)
#     #dsout.chunk({'Nc':-1,'time':1}).to_zarr(outfile, mode=mode, append_dim=append_dim, compute=True)
#     dsout.chunk({'Nc':2000}).to_zarr(outfile, mode=mode, append_dim=append_dim, compute=True)

#     mode='a'
#     append_dim='time'
    
#     # Clear memory
#     client.restart()
   

/opt/conda/lib/python3.10/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 9.79 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


RuntimeError: NetCDF: Can't open HDF5 attribute

ValueError: Array chunk size or shape is unknown. Possible solution with x.compute_chunk_sizes()